# BO runs

In [1]:
import torch
from botorch.models import FixedNoiseGP, SingleTaskGP
from gpytorch.kernels import ScaleKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.acquisition.analytic import ExpectedImprovement
import numpy as np
import pickle
import sys
import time

load data from `prepare_Xy.ipynb`

In [2]:
X = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['X']
y = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['y']
y = np.reshape(y, (np.size(y), 1)) # for the GP
nb_data = np.size(y)
nb_data

69839

convert to torch tensors

In [3]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)

In [4]:
X.size()

torch.Size([69839, 12])

In [5]:
y.size()

torch.Size([69839, 1])

In [6]:
X_unsqueezed = X.unsqueeze(1)

In [7]:
batch_size = 10000
acquisition_values = np.zeros((nb_data))
acquisition_values[:] = np.NaN
nb_batches = nb_data // batch_size
for ba in range(nb_batches+1):
    id_start = ba * batch_size
    id_end   = id_start + batch_size
    if id_end > nb_data:
        id_end = nb_data
    acquisition_values[id_start:id_end] = range(id_start, id_end)
    
np.sum(np.isnan(acquisition_values))

0

In [8]:
nb_data

69839

number of COFs for initialization

In [9]:
nb_COFs_initialization = 10

In [10]:
def bo_run(nb_iterations):
    assert nb_iterations > nb_COFs_initialization
    
    # select initial COFs for training data randomly.
    # idea is to keep populating this ids_acquired and return it for analysis.
    ids_acquired = np.random.choice(np.arange((nb_data)), size=nb_COFs_initialization, replace=False)

    # initialize acquired y, since it requires normalization
    y_acquired = y[ids_acquired]
    # standardize outputs
    y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)
    
    for i in range(nb_COFs_initialization, nb_iterations):
        print("iteration:", i)
        # construct and fit GP model
        model = SingleTaskGP(X[ids_acquired, :], y_acquired)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_model(mll)

        # set up acquisition function
        acquisition_function = ExpectedImprovement(model, best_f=y_acquired.max().item())
        
        # compute aquisition function at each COF in the database. need to do in batches to avoid mem issues
        batch_size = 20000
        acquisition_values = torch.zeros((nb_data))
        acquisition_values[:] = np.NaN # for safety
        nb_batches = nb_data // batch_size
        for ba in range(nb_batches+1):
            id_start = ba * batch_size
            id_end   = id_start + batch_size
            if id_end > nb_data:
                id_end = nb_data
            acquisition_values[id_start:id_end] = acquisition_function.forward(X_unsqueezed[id_start:id_end])
#         acquisition_values = acquisition_function.forward(X_unsqueezed)
        assert acquisition_values.isnan().sum().item() == 0 # so that all are filled properly.
        del acquisition_function

        # select COF to acquire with maximal aquisition value, which is not in the acquired set already
        ids_sorted_by_aquisition = acquisition_values.argsort(descending=True)
        for id_max_aquisition_all in ids_sorted_by_aquisition:
            if not id_max_aquisition_all.item() in ids_acquired:
                id_max_aquisition = id_max_aquisition_all.item()
                break

        # acquire this COF
        ids_acquired = np.concatenate((ids_acquired, [id_max_aquisition]))
        assert np.size(ids_acquired) == i + 1

        # update y aquired; start over to normalize properly
        del y_acquired
        y_acquired = y[ids_acquired, :] # start over to normalize y properly
        y_acquired = (y_acquired - torch.mean(y_acquired)) / torch.std(y_acquired)

        print("\tacquired COF", id_max_aquisition, "with y = ", y[id_max_aquisition].item())
        print("\tbest y acquired:", y[ids_acquired].max().item())
        
        del model
        del mll
        del acquisition_values
        
    assert np.size(ids_acquired) == nb_iterations
    return ids_acquired

`ids_acquired[r, i]` will give ID of COF acquired during iteration `i` from run `r`.

In [ ]:
bo_res = dict()
bo_res['nb_runs']       = 10
bo_res['nb_iterations'] = 250
bo_res['ids_acquired'] = []
for r in range(bo_res['nb_runs']):
    print("\n\nRUN", r)
    t0 = time.time()
    ids_acquired = bo_run(bo_res['nb_iterations'])
    bo_res['ids_acquired'].append(ids_acquired)
    print("took time t = ", (time.time() - t0) / 60, "min")



RUN 0
iteration: 10
	acquired COF 38686 with y =  181.997321887
	best y acquired: 181.997321887
iteration: 11
	acquired COF 14592 with y =  173.52788820799998
	best y acquired: 181.997321887
iteration: 12
	acquired COF 13446 with y =  156.582607879
	best y acquired: 181.997321887
iteration: 13
	acquired COF 35153 with y =  185.721713331
	best y acquired: 185.721713331
iteration: 14
	acquired COF 442 with y =  177.769833484
	best y acquired: 185.721713331
iteration: 15
	acquired COF 2782 with y =  190.660502314
	best y acquired: 190.660502314
iteration: 16
	acquired COF 776 with y =  164.249792893
	best y acquired: 190.660502314
iteration: 17
	acquired COF 19351 with y =  191.120614308
	best y acquired: 191.120614308
iteration: 18
	acquired COF 27950 with y =  177.605961385
	best y acquired: 191.120614308
iteration: 19
	acquired COF 21852 with y =  189.50649556599998
	best y acquired: 191.120614308
iteration: 20
	acquired COF 14294 with y =  173.051146072
	best y acquired: 191.1206143

In [ ]:
with open('bo_results.pkl', 'wb') as file:
    pickle.dump(bo_res, file)